In [1]:
# script made on 16gb of ram
import os
import io
import os.path
import random
import pickle
import zipfile
from math import ceil
from copy import deepcopy
import numpy as np
from tqdm import tqdm
import pandas as pd
import tensorflow as tf      #the progress bar
import en_core_web_sm as en  #from the spaCy library, https://spacy.io/usage/

In [2]:
filename = "./data/train.csv"
output = "./data/output/"

In [3]:
# enb_path = 'input/embeddings/wiki-news-300d-1M/' + 'wiki-news-300d-1M.vec'
# enb_path = 'input/embeddings/GoogleNews-vectors-negative300/'
# enb_path = 'input/embeddings/paragram_300_sl999/'
enb_path = "./data/embeddings/glove.840B.300d/glove.840B.300d.txt"

In [4]:
#global variables to save memory
# _t is for tokenized
embeddings = []      #np array of embeddings
vocab = []          #the vocabulary from the enbeding file
questions_t = []    #every question tokenized in an array
text_t = []         #all the questions concatenated and tokenized
word_to_index = {}
index_to_word = {}

In [5]:
def tokenize_questions(text,nr_to_delete = 0):
    nlp = en.load()      #load the tokenizer
    
    global questions_t
    global vocab         #every unique word in the appearance order  
    global text_t       #every token in the original order
    
    text_t = [] 
    vocab = [] 
    questions_t = []
    
    del text  #delete the original text so we save memory
    bar  = tqdm(total = len(batches))
    
    for batch in batches:  #tqdm is the progress bar
        tokens = nlp(batch)
        batch_tokenized = []
        for token in tokens:
            word = token.string.strip()
            text_t.append(word)
            batch_tokenized.append(word)
        questions_t.append(batch_tokenized)
        bar.update(1)
    bar.close()

In [6]:
train_df = pd.read_csv(filename)    #test_t is for now a dataFrame

In [7]:
y_train = train_df.values[:,2].tolist()

In [8]:
# loadig the qustions tokenized, and all qustions concatenated tokenized
if not os.path.isfile(output + "questions_t"):
    tokenize_questions( train_df.values[:,1].tolist() )
    pickle.dump(questions_t, open(output + "questions_t",'wb') )
    pickle.dump(text_t, open(output + "text_t",'wb') )
else:
    print("Loading data...")
    questions_t = pickle.load( open(output + "questions_t", "rb" ) )
    text_t = pickle.load( open(output + "text_t", "rb" ) )
    print("Nr of questions: ",len(questions_t))
    print("Nr of tokens:    ",len(text_t))
    
print('The fist question tokenized: ', questions_t[0][:10])

Loading data...
Nr of questions:  1306122
Nr of tokens:     19020768
The fist question tokenized:  ['How', 'did', 'Quebec', 'nationalists', 'see', 'their', 'province', 'as', 'a', 'nation']


In [9]:
def load_embeddings(enb_path):
    global embeddings
    global vocab
    
    embeddings = []
    vocab = []
    
    print("Loading embeddings...")
    
    with open(enb_path, 'r', encoding ='utf-8') as file:
    
        num_lines = sum(1 for line in file)
        bar  = tqdm(total = num_lines)
    
        file.seek(0)
    
        vocab = []
        embeddings = []      
        for line in file:
            bar.update(1)
            array = line[:-1].split(' ')
            vocab.append(str(array[0]))
            embeddings.append(np.array(array[1:]).astype('float32'))      
        bar.close()
        
    embeddings = np.stack(embeddings)

In [10]:
if not os.path.isfile(output + "embeddings"):
    load_embeddings(enb_path)
    pickle.dump(embeddings,open( output + "embeddings",'wb'))
    pickle.dump(vocab,open( output + "vocabulary",'wb'))
else:
    print("Loading embeddings...")
    embeddings = pickle.load( open(output + "embeddings", "rb" ) )
    vocab = pickle.load( open( output + "vocabulary", "rb" ) )
print('Embedings size:  ',embeddings.shape)
print('Vocabulary size: ', len(vocab))

print('\nThe word "'+ vocab[2]+'" with his enmeding: \n' , embeddings[2][:5], "... ", embeddings[2][-5:] )

Loading embeddings...
Embedings size:   (2196017, 300)
Vocabulary size:  2196017

The word "the" with his enmeding: 
 [ 0.27204  -0.06203  -0.1884    0.023225 -0.018158] ...  [-0.018168  0.11407   0.13015  -0.18317   0.1323  ]


In [11]:
def tokens_to_index():
    global word_to_index
    global index_to_word
    global questions_t
    global vocab
    
    word_to_index = {}
    index_to_word = {}
    
    for idx, word in enumerate(vocab): #create the two dictionaries
        word_to_index[word] = idx
        index_to_word[idx] = word
        
    for batch in questions_t: #chage the input from strings to ids
        if isinstance(batch, int):
            print(batch)
        for idx,token in enumerate(batch): #replace the text with the word indexes
             batch[idx] = word_to_index.get(token, 132037) #if the token is not in vocab put "UNK"

In [13]:
def initialze_padding():
    global vocab         #we need to add the padding word to our vocabulary
    global embeddings    #we need to add the padding word to our embedding matrix
    global word_to_index
    global index_to_word
    
    print("Adding the padding char to vocab and embeddings...")
    vocab = ['/pad']+vocab
    pad_emb = []
    for i in range(len(embeddings[0])):
        pad_emb.append(0)
    pad_emb = np.array(pad_emb)
    embeddings = np.vstack([pad_emb,embeddings])
    
    tokens_to_index()
    

def pad_questions(data, max_len = 60):
    if not (vocab[0] == "/pad" and index_to_word[0] == "/pad"):
        initialze_padding()
    
    
    print("Padding questions...")
    data_aux = deepcopy(data)
    
    padded_count = 0
    cut_count = 0
    for idx in tqdm(range(len(data_aux))):
        question_len = len(data_aux[idx])
        if question_len <= max_len:
            padded_count += 1
            for i in range(question_len, max_len):
                data_aux[idx].append(0)
        elif question_len > max_len:
            cut_count += 1
            data_aux[idx] = data_aux[idx][:max_len]
        
        if len(data_aux[idx])>max_len:
            print(question_len)
            print(question)
            break
            
    print("Padded :", padded_count)
    print("Cut    :", cut_count)
    return data_aux

class Batch():
    def __init__(self, data, labels, size = 64):
        self.data = data
        del data
        self.labels = labels
        self.size = size
        self.index = 0
    
    def __next__(self):
        
        if (len(self.data) <= self.index):
            raise StopIteration()
        
        self.index = self.index + self.size    
        
        return self.data[self.index-self.size:self.index],self.labels[self.index-self.size:self.index]        
    
    def __len__(self):
        return ceil(len(self.data)/self.size)
    
    def reset(self):
        self.index = 0
    
    def __iter__(self):
        return self

In [14]:
questions_t = pad_questions(questions_t)

Adding the padding char to vocab and embeddings...
Padding questions...


100%|████████████████████████████████████████████████████████████████████| 1306122/1306122 [00:04<00:00, 265729.84it/s]


Padded : 1305887
Cut    : 235


In [17]:
def embedding_lookup(batch):
    global embeddings
    
    batch_embedding = []
    
    for int_question in batch:
        question_embedding = []

        for int_word in int_question:
            try:
                question_embedding.append(embeddings[int_word])
            except Exception:
                print(int_word)
        
        enb = np.stack(question_embedding)
        batch_embedding.append(enb)
    for el in batch_embedding:
        if len(el)!=60:
            print(el.shape)
    return np.stack(batch_embedding)

In [18]:
print('The word UNK to index       :', word_to_index['UNK'])
print('The intex for UNK to string :', index_to_word[132037])
print('The shape of enbedings      :', embeddings.shape)
print('The first 5words from vocab :',vocab[:10])

The word UNK to index       : 132037
The intex for UNK to string : UNK
The shape of enbedings      : (2196018, 300)
The first 5words from vocab : ['/pad', ',', '.', 'the', 'and', 'to', 'of', 'a', 'in', '"']


In [19]:
def linear_regresion():
    
    def batch_matmul_right(X,w):
        mult = []
        for x in tf.unstack(X,axis=0):
            mult.append(tf.matmul(x,w))
        return tf.stack(mult)
    
    def batch_matmul_left(w,X):
        mult = []
        for x in tf.unstack(X,axis=0):
            mult.append(tf.matmul(w,x))
        return tf.stack(mult)
    
    x = tf.placeholder(tf.float32,shape=(64, 60, 300), name = 'x')
    y = tf.placeholder(tf.float32,shape=(64, ),name='y')
    
    with tf.variable_scope('lreg') as scope:
        
        w = tf.Variable(tf.random_uniform((300,1), -1, 1), name='W')
        print(x.shape, w.shape)
        W = tf.stack([w]*64)
        #mul = tf.matmul(w,x)
        mul = batch_matmul_right(x,w)
        print(mul.shape)
        w2 = tf.Variable(tf.random_uniform((1,60), -1, 1), name='W2')
        mul2 = batch_matmul_left(w2,mul)
        mul2 = tf.reshape(mul2,(64, ))
        print(mul2.shape)
        loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=mul2)        
#         linear1 = tf.nn.relu(tf.matmul(x, w))

#         w2 = tf.Variable(tf.random_uniform((1,None), -1, 1), name='W')
        
#         y_pred = tf.matmul(w2, linear1)
        
#         loss = tf.square(y_pred - y)
        
    return x, y, loss

In [21]:
def run(batches):
    global questions_t
    global y_train
    global embeddigs
    
    
    x, y, loss = linear_regresion()
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
    
    
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        
        average_loss = 0
        for batch in tqdm(batches):
            batch, label =  batch
            batch = embedding_lookup( batch )
            label = np.array(label)
            _ ,batch_loss = session.run([optimizer, loss], {x:batch, y :label})
            average_loss += batch_loss
        print("Avg loss :",average_loss/len(batches))
    
    return y_pred_batch

In [23]:
batches = Batch(questions_t, y_train, 64)

In [22]:
run(batches)

(64, 60, 300) (300, 1)
(64, 60, 1)
(64,)


100%|███████████████████████████████████████████████████████████████████████████▉| 20407/20409 [04:45<00:00, 69.57it/s]

ValueError: Cannot feed value of shape (10, 60, 300) for Tensor 'x:0', which has shape '(64, 60, 300)'

100%|███████████████████████████████████████████████████████████████████████████▉| 20407/20409 [05:00<00:00, 69.57it/s]